# Hotspots Visualizations

In [ ]:
import pandas as pd
asthma_df = pd.read_csv("../code and data/NYC EH Data Portal - Adults with asthma (past 12 months) (full table).csv")
print(asthma_df['Geography'].unique())

['Kingsbridge - Riverdale' 'Northeast Bronx' 'Fordham - Bronx Pk'
 'Pelham - Throgs Neck' 'Greenpoint' 'Downtown - Heights - Slope'
 'Bedford Stuyvesant - Crown Heights' 'East New York' 'Sunset Park'
 'Borough Park' 'East Flatbush - Flatbush' 'Canarsie - Flatlands'
 'Bensonhurst - Bay Ridge' 'Coney Island - Sheepshead Bay'
 'Williamsburg - Bushwick' 'Washington Heights'
 'Central Harlem - Morningside Heights' 'East Harlem' 'Upper West Side'
 'Long Island City - Astoria' 'West Queens' 'Flushing - Clearview'
 'Ridgewood - Forest Hills' 'Southwest Queens' 'Jamaica'
 'Southeast Queens' 'Rockaways' 'Upper East Side-Gramercy'
 'Chelsea-Village  ' 'Union Square-Lower Manhattan'
 'Bayside Little Neck-Fresh Meadows' 'Northern SI' 'Southern SI'
 'South Bronx' 'Bronx' 'Brooklyn' 'Manhattan' 'Queens' 'Staten Island'
 'New York City']


### Interactive Hotspot Visualization with Rescaling and Highlight Features

In [ ]:
import geopandas as gpd
import pandas as pd
import json
from bokeh.io import show, output_file
from bokeh.models import (
    GeoJSONDataSource, ColumnDataSource, WMTSTileSource,
    HoverTool, LinearColorMapper, ColorBar, Select, CustomJS, 
    Button, LinearInterpolator
)
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.palettes import RdYlGn, Blues9

# -------------------------------
# Load districts and reproject
# -------------------------------
gdf = gpd.read_file("../code and data/uhf34.geojson")
gdf = gdf.to_crs(epsg=3857)
centroids = gdf.geometry.centroid
gdf["centroid_x"] = centroids.x
gdf["centroid_y"] = centroids.y

# -------------------------------
# List of poorest neighborhoods
# -------------------------------
poor_districts = [
    "South Bronx",
    "East Flatbush - Flatbush",
    "Rockaways",
    "Central Harlem - Morningside Heights",
    "Fordham - Bronx Pk",
    "Pelham - Throgs Neck",
    "Canarsie - Flatlands",
    "East New York", 
    "Brownsville", 
    "East Harlem", 
    "Far Rockaway", 
    "Southeast Queens",
    "Southwest Queens",
    "Jamaica",
    "Central Harlem - Morningside Heights",
    "Bedford Stuyvesant - Crown Heights",
    "Washington Heights",
    "Coney Island - Sheepshead Bay",
    "Williamsburg - Bushwick",
    "Bronx",
    "Brooklyn"
]
gdf['is_poor'] = gdf['GEONAME'].isin(poor_districts)

# -------------------------------
# PM2.5 Data
# -------------------------------
pm_df = pd.read_csv("../code and data/NYC EH Data Portal - Fine particles (PM 2.5) (full table).csv")
pm_df = pm_df[pm_df["TimePeriod"].between(2009, 2023)]

# -------------------------------
# Asthma Data
# -------------------------------
asthma_df = pd.read_csv("../code and data/NYC EH Data Portal - Adults with asthma (past 12 months) (full table).csv")
asthma_df = asthma_df[asthma_df["TimePeriod"].between(2009, 2022)]
asthma_df["Number"] = asthma_df["Number"].str.replace(",", "").str.extract("(\d+)").astype(float)

# -------------------------------
# Create GeoJSON and points by year
# -------------------------------
geojson_by_year = {}
asthma_data_by_year = {}
asthma_minmax_by_year = {}
pm_minmax_by_year = {}
years = sorted(pm_df["TimePeriod"].dropna().unique())
years = [year for year in years if year != 2023]

for year in years:
    pm_year = pm_df[pm_df["TimePeriod"] == year]
    asthma_year = asthma_df[asthma_df["TimePeriod"] == year]

    merged = gdf.copy()
    merged["PM25"] = merged["GEONAME"].map(pm_year.set_index("Geography")["Annual mean mcg/m3"].to_dict())
    merged["Asma"] = merged["GEONAME"].map(asthma_year.set_index("Geography")["Number"].to_dict())
    merged["fill_alpha"] = 0.7
    merged = merged.dropna(subset=["PM25", "Asma"])

    geojson_by_year[str(year)] = json.loads(merged.to_json())

    # For asthma values
    asthma_values = merged["Asma"].fillna(0).tolist()
    if asthma_values:
        asthma_minmax_by_year[str(year)] = (min(asthma_values), max(asthma_values))
    else:
        asthma_minmax_by_year[str(year)] = (0, 0)

    asthma_data_by_year[str(year)] = {
        "x": merged["centroid_x"].tolist(),
        "y": merged["centroid_y"].tolist(),
        "value": asthma_values,
        "name": merged["GEONAME"].tolist()
    }
    
    # For PM2.5 values
    pm_values = merged["PM25"].fillna(0).tolist()
    if pm_values:
        pm_minmax_by_year[str(year)] = (min(pm_values), max(pm_values))
    else:
        pm_minmax_by_year[str(year)] = (0, 0)

# -------------------------------
# Initialization
# -------------------------------
initial_year = str(years[0])
geo_source = GeoJSONDataSource(geojson=json.dumps(geojson_by_year[initial_year]))
asthma_source = ColumnDataSource(asthma_data_by_year[initial_year])
pm_source = ColumnDataSource({"x": [], "y": [], "value": []})  # Data source for PM2.5

# -------------------------------
# Create figure
# -------------------------------
bounds = gdf.total_bounds
p = figure(
    title=f"PM2.5 Annual Average - {initial_year}",
    x_axis_type="mercator", y_axis_type="mercator",
    x_range=(bounds[0] - 10000, bounds[2] + 10000),
    y_range=(bounds[1] - 10000, bounds[3] + 10000),
    width=900, height=600
)
tile_provider = WMTSTileSource(url='https://tile.openstreetmap.org/{Z}/{X}/{Y}.png')
p.add_tile(tile_provider)

# -------------------------------
# Color mapping for PM2.5
# -------------------------------
pm_color_mapper = LinearColorMapper(palette=RdYlGn[9],
                                    low=pm_minmax_by_year[initial_year][0],
                                    high=pm_minmax_by_year[initial_year][1])

patches = p.patches("xs", "ys", source=geo_source,
                    fill_color={'field': 'PM25', 'transform': pm_color_mapper},
                    line_color="black", line_width=0.5,
                    fill_alpha={'field': 'fill_alpha'})

# -------------------------------
# Color and size mapping for asthma
# -------------------------------
asthma_circle_mapper = LinearColorMapper(palette=Blues9[1:5][::-1],  # Remove white, reverse palette
                                         low=asthma_minmax_by_year[initial_year][0],
                                         high=asthma_minmax_by_year[initial_year][1])

size_mapper = LinearInterpolator(
    x=[0, 20000],
    y=[5, 30]
)

circles = p.scatter(x='x', y='y', source=asthma_source,
                   size={'field': 'value', 'transform': size_mapper},
                   fill_color={'field': 'value', 'transform': asthma_circle_mapper},
                   fill_alpha=0.6, line_color="white")

# -------------------------------
# Buttons
# -------------------------------
highlight_button = Button(label="Highlight Poor Neighborhoods", button_type="primary")
highlight_button.js_on_click(CustomJS(args=dict(source=geo_source), code=""" 
    const data = JSON.parse(source.geojson);
    data.features.forEach(f => {
        f.properties.fill_alpha = f.properties.is_poor ? 0.9 : 0.2;
    });
    source.geojson = JSON.stringify(data);
"""))

reset_button = Button(label="Reset Opacity", button_type="primary")
reset_button.js_on_click(CustomJS(args=dict(source=geo_source), code=""" 
    const data = JSON.parse(source.geojson);
    data.features.forEach(f => {
        f.properties.fill_alpha = 0.7;
    });
    source.geojson = JSON.stringify(data);
"""))

# -------------------------------
# Year dropdown with dynamic updates
# -------------------------------
select = Select(title="Choose Reference Year", value=initial_year, options=[str(y) for y in years])
select.js_on_change("value", CustomJS(args=dict(
    source=geo_source,
    asthma_source=asthma_source,
    geojsons=geojson_by_year,
    asthma_data=asthma_data_by_year,
    plot=p,
    asthma_mapper=asthma_circle_mapper,
    minmax=asthma_minmax_by_year,
    pm_mapper=pm_color_mapper,
    pm_minmax=pm_minmax_by_year
), code=""" 
    const year = cb_obj.value;

    source.geojson = JSON.stringify(geojsons[year]);
    asthma_source.data = asthma_data[year];

    const [low, high] = minmax[year];
    asthma_mapper.low = low;
    asthma_mapper.high = high;

    // Update PM2.5 color mapper
    const pmLow = pm_minmax[year][0];
    const pmHigh = pm_minmax[year][1];
    pm_mapper.low = pmLow;
    pm_mapper.high = pmHigh;

    plot.title.text = `PM2.5 Annual Average - ${year}`;
"""))

# -------------------------------
# Tooltips
# -------------------------------
p.add_tools(HoverTool(renderers=[patches], tooltips=[("District", "@GEONAME"), ("PM2.5 (µg/m³)", "@PM25")]))
p.add_tools(HoverTool(renderers=[circles], tooltips=[("District", "@name"), ("Asthma Cases", "@value")]))

# -------------------------------
# Color bars
# -------------------------------
color_bar = ColorBar(color_mapper=pm_color_mapper, label_standoff=12, 
                     location=(0, 0), title="PM2.5 Levels")
p.add_layout(color_bar, 'right')

asthma_color_bar = ColorBar(color_mapper=asthma_circle_mapper,
                            label_standoff=12,
                            location=(0, 0),
                            title="Asthma Cases")
p.add_layout(asthma_color_bar, 'right')

# -------------------------------
# Final layout and display
# -------------------------------
# output_file("hotspot_resclaing.html")
layout = column(select, row(highlight_button, reset_button), p)
show(layout)


### Interactive Hotspot Visualization with Highlight Feature and Without Rescaling

In [9]:
import geopandas as gpd
import pandas as pd
import json
from bokeh.io import show, output_file
from bokeh.models import (
    GeoJSONDataSource, ColumnDataSource, WMTSTileSource,
    HoverTool, LinearColorMapper, ColorBar, Select, CustomJS, 
    Button, LinearInterpolator
)
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.palettes import RdYlGn, Blues9

# -------------------------------
# Load districts and reproject
# -------------------------------
gdf = gpd.read_file("../code and data/uhf34.geojson")
gdf = gdf.to_crs(epsg=3857)
centroids = gdf.geometry.centroid
gdf["centroid_x"] = centroids.x
gdf["centroid_y"] = centroids.y

# -------------------------------
# List of the poorest neighborhoods
# -------------------------------
poor_districts = [
    "South Bronx", "East Flatbush - Flatbush", "Rockaways",
    "Central Harlem - Morningside Heights", "Fordham - Bronx Pk",
    "Pelham - Throgs Neck", "Canarsie - Flatlands", "East New York", 
    "Brownsville", "East Harlem", "Far Rockaway", "Southeast Queens",
    "Southwest Queens", "Jamaica", "Central Harlem - Morningside Heights",
    "Bedford Stuyvesant - Crown Heights", "Washington Heights",
    "Coney Island - Sheepshead Bay", "Williamsburg - Bushwick",
    "Bronx", "Brooklyn"
]
gdf['is_poor'] = gdf['GEONAME'].isin(poor_districts)

# -------------------------------
# PM2.5 data
# -------------------------------
pm_df = pd.read_csv("../code and data/NYC EH Data Portal - Fine particles (PM 2.5) (full table).csv")
pm_df = pm_df[pm_df["TimePeriod"].between(2009, 2023)]

# -------------------------------
# Asthma data
# -------------------------------
asthma_df = pd.read_csv("../code and data/NYC EH Data Portal - Adults with asthma (past 12 months) (full table).csv")
asthma_df = asthma_df[asthma_df["TimePeriod"].between(2009, 2023)]
asthma_df["Number"] = asthma_df["Number"].str.replace(",", "").str.extract("(\d+)").astype(float)

# -------------------------------
# Create GeoJSON and per-year point data
# -------------------------------
geojson_by_year = {}
asthma_data_by_year = {}
asthma_minmax_by_year = {}
pm_minmax_by_year = {}
years = sorted(pm_df["TimePeriod"].dropna().unique())
years = [year for year in years if year != 2023]

for year in years:
    pm_year = pm_df[pm_df["TimePeriod"] == year]
    asthma_year = asthma_df[asthma_df["TimePeriod"] == year]

    merged = gdf.copy()
    merged["PM25"] = merged["GEONAME"].map(pm_year.set_index("Geography")["Annual mean mcg/m3"].to_dict())
    merged["Asma"] = merged["GEONAME"].map(asthma_year.set_index("Geography")["Number"].to_dict())
    merged["fill_alpha"] = 0.7
    merged = merged.dropna(subset=["PM25", "Asma"])

    geojson_by_year[str(year)] = json.loads(merged.to_json())

    # Asthma values
    asthma_values = merged["Asma"].fillna(0).tolist()
    if asthma_values:
        asthma_minmax_by_year[str(year)] = (min(asthma_values), max(asthma_values))
    else:
        asthma_minmax_by_year[str(year)] = (0, 0)

    asthma_data_by_year[str(year)] = {
        "x": merged["centroid_x"].tolist(),
        "y": merged["centroid_y"].tolist(),
        "value": asthma_values,
        "name": merged["GEONAME"].tolist()
    }
    
    # PM2.5 values
    pm_values = merged["PM25"].fillna(0).tolist()
    if pm_values:
        pm_minmax_by_year[str(year)] = (min(pm_values), max(pm_values))
    else:
        pm_minmax_by_year[str(year)] = (0, 0) 

# -------------------------------
# Initialization
# -------------------------------
initial_year = str(years[0])
geo_source = GeoJSONDataSource(geojson=json.dumps(geojson_by_year[initial_year]))
asthma_source = ColumnDataSource(asthma_data_by_year[initial_year])
initial_low, initial_high = asthma_minmax_by_year[initial_year]

# -------------------------------
# Create figure
# -------------------------------
bounds = gdf.total_bounds
p = figure(
    title=f"PM2.5 Annual Average - {initial_year}",
    x_axis_type="mercator", y_axis_type="mercator",
    x_range=(bounds[0] - 10000, bounds[2] + 10000),
    y_range=(bounds[1] - 10000, bounds[3] + 10000),
    width=900, height=600
)
tile_provider = WMTSTileSource(url='https://tile.openstreetmap.org/{Z}/{X}/{Y}.png')
p.add_tile(tile_provider)

# -------------------------------
# PM2.5 color mapping
# -------------------------------
color_mapper = LinearColorMapper(palette=RdYlGn[9],
                                 low=pm_df["Annual mean mcg/m3"].min(),
                                 high=pm_df["Annual mean mcg/m3"].max())

patches = p.patches("xs", "ys", source=geo_source,
                    fill_color={'field': 'PM25', 'transform': color_mapper},
                    line_color="black", line_width=0.5,
                    fill_alpha={'field': 'fill_alpha'})

# -------------------------------
# Asthma color and size mapping
# -------------------------------
asthma_circle_mapper = LinearColorMapper(palette=Blues9[1:5][::-1],  # Remove white and reverse palette
                                         low=asthma_minmax_by_year[initial_year][0],
                                         high=asthma_minmax_by_year[initial_year][1])

size_mapper = LinearInterpolator(
    x=[0, 20000],
    y=[5, 30]
)

circles = p.scatter(x='x', y='y', source=asthma_source,
                   size={'field': 'value', 'transform': size_mapper},
                   fill_color={'field': 'value', 'transform': asthma_circle_mapper},
                   fill_alpha=0.6, line_color="white")

# -------------------------------
# Buttons
# -------------------------------
highlight_button = Button(label="Highlight Poor Neighborhoods", button_type="primary")
highlight_button.js_on_click(CustomJS(args=dict(source=geo_source), code=""" 
    const data = JSON.parse(source.geojson);
    data.features.forEach(f => {
        f.properties.fill_alpha = f.properties.is_poor ? 0.9 : 0.2;
    });
    source.geojson = JSON.stringify(data);
"""))

reset_button = Button(label="Reset Opacity", button_type="primary")
reset_button.js_on_click(CustomJS(args=dict(source=geo_source), code=""" 
    const data = JSON.parse(source.geojson);
    data.features.forEach(f => {
        f.properties.fill_alpha = 0.7;
    });
    source.geojson = JSON.stringify(data);
"""))

# -------------------------------
# Year dropdown with color mapper update
# -------------------------------
select = Select(title="Choose Reference Year", value=initial_year, options=[str(y) for y in years])
select.js_on_change("value", CustomJS(args=dict(
    source=geo_source,
    asthma_source=asthma_source,
    geojsons=geojson_by_year,
    asthma_data=asthma_data_by_year,
    plot=p,
    asthma_mapper=asthma_circle_mapper,
    minmax=asthma_minmax_by_year
), code="""
    const year = cb_obj.value;

    source.geojson = JSON.stringify(geojsons[year]);
    asthma_source.data = asthma_data[year];

    const [low, high] = minmax[year];
    asthma_mapper.low = low;
    asthma_mapper.high = high;

    plot.title.text = `PM2.5 Annual Average - ${year}`;
"""))

# -------------------------------
# Tooltips
# -------------------------------
p.add_tools(HoverTool(renderers=[patches], tooltips=[
    ("District", "@GEONAME"),
    ("PM2.5 (µg/m³)", "@PM25")
]))

p.add_tools(HoverTool(renderers=[circles], tooltips=[
    ("District", "@name"),
    ("Asthma Cases", "@value")
]))

# -------------------------------
# Colorbars
# -------------------------------
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, 
                     location=(0, 0), title="PM2.5 Levels")
p.add_layout(color_bar, 'right')

asthma_color_bar = ColorBar(color_mapper=asthma_circle_mapper,
                            label_standoff=12,
                            location=(0, 0),
                            title="Asthma Cases")
p.add_layout(asthma_color_bar, 'right')

# -------------------------------
# Final layout
# -------------------------------
# output_file("hotspot_without_rescaling.html")
layout = column(select, row(highlight_button, reset_button), p)
show(layout)
